In [2]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('t2.jpg')

width, height = 480, 620

def nothing(x):
    pass

cv2.namedWindow("Trackbars")
cv2.resizeWindow("Trackbars", 360, 240)
cv2.createTrackbar("Threshold1", "Trackbars", 200,255, nothing)
cv2.createTrackbar("Threshold2", "Trackbars", 200, 255, nothing)

def biggestContour(contours):
    biggest = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 5000:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02 * peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    return biggest,max_area

 
def reorder(myPoints):
 
    myPoints = myPoints.reshape((4, 2))
    myPointsNew = np.zeros((4, 1, 2), dtype=np.int32)
    add = myPoints.sum(1)
    
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] =myPoints[np.argmax(add)]
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1] =myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
 
    return myPointsNew
 

while True:
    img = cv2.resize(img, (width, height))
    img_copy = img.copy()
    img_final = img.copy()
    img_0 = img.copy()
    

    Gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    Gray_blur = cv2.GaussianBlur(Gray,(5,5), 1)

    Threshold1 = cv2.getTrackbarPos("Threshold1", "Trackbars")
    Threshold2 = cv2.getTrackbarPos("Threshold2", "Trackbars")
    Edges = cv2.Canny(Gray_blur,Threshold1,Threshold2)
    kernel = np.ones((5, 5))

    imgDial = cv2.dilate(Edges, kernel, iterations=2) # APPLY DILATION
    imgThreshold = cv2.erode(imgDial, kernel, iterations=1)  # APPLY EROSION
 
    # contour
    contour, heirarchy = cv2.findContours(Edges, cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
    # cv2.drawContours(img_, contour, -1,(0,0,255), 5)

    biggest, maxArea = biggestContour(contour) # FIND THE BIGGEST CONTOUR

    if biggest.size != 0:
        biggest=reorder(biggest)
        cv2.drawContours(img_copy, biggest, -1, (0, 255, 0), 20) # DRAW THE BIGGEST CONTOUR
        
        pts1 = np.float32(biggest) # PREPARE POINTS FOR WARP
        pts2 = np.float32([[0, 0],[width, 0], [0, height],[width, height]]) # PREPARE POINTS FOR WARP
        matrix = cv2.getPerspectiveTransform(pts1, pts2)
        img_final = cv2.warpPerspective(img, matrix, (width, height))

    cv2.imshow("Original", img_copy)
    cv2.imshow("Final Image", img_final)
    cv2.imshow("Object Edge", Edges)

    if cv2.waitKey(1) == ord('q'):
        break
        


cv2.destroyAllWindows()